In [299]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision.transforms import Normalize
from ax import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting
import torch.nn as nn
import logging



class CNNTrainer:
    def __init__(self, train_data, batch_size=64, model=None):
        self.model = model
        # Load the training data
        self.train_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
        
    def compute_input_size(self, params):
        # Add BatchNorm2d layer to standardize input data
        x = nn.BatchNorm2d(3)(torch.zeros([1,3,32,32]))
        # Pass the input through the sequence of layers
        for layer in [
                nn.Conv2d(in_channels=3, 
                          out_channels=params.get('num_filters1'), 
                          kernel_size=params.get('filter_size1')),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3),
                nn.Conv2d(in_channels=params.get('num_filters1'),
                          out_channels=params.get('num_filters2'),
                          kernel_size=params.get('filter_size2')),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
                nn.Conv2d(in_channels=params.get('num_filters2'),
                          out_channels=params.get('num_filters3'),
                          kernel_size=params.get('filter_size3')),
                nn.ReLU(),
                nn.Flatten()]:
            x = layer(x)
        input_size = x.shape[0] * x.shape[1]
        return input_size

    def build_model(self, params):
        linear_input = self.compute_input_size(params) 
        # Define the CNN architecture based on the given parameters
        model = nn.Sequential(
            nn.BatchNorm2d(3),  # Add BatchNorm2d layer to standardize input data,
            nn.Conv2d(in_channels=3, 
                      out_channels=params.get('num_filters1'), 
                      kernel_size=params.get('filter_size1')),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=params.get('num_filters1'),
                      out_channels=params.get('num_filters2'),
                      kernel_size=params.get('filter_size2')),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=params.get('num_filters2'),
                      out_channels=params.get('num_filters3'),
                      kernel_size=params.get('filter_size3')),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(linear_input, 10)
        )
        return model
        
        
    def fit(self, model, epochs=1):
        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Train the model
        for epoch in range(epochs):
            print("Running epoch ", epoch)
            total, correct = 0,0
            for images, labels in self.train_loader:
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            accuracy = correct / total  
            print(f"ACC for epoch {epoch}: ", accuracy)
        return model
        
class CNNPredictor:
    def __init__(self, model):
        self.model = model
        
    def predict(self, test_data):
        # Evaluate the model on the validation set
        data_loader = data.DataLoader(test_data, batch_size=64)
        
        # Calculate validation accuracy
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in data_loader:
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        return labels, accuracy

class CNNOptimizer:
    def __init__(self, search_space, train_data, val_data, steps=20, epochs=1):
        self.epochs = epochs
        self.steps = steps
        self.search_space = search_space
        self.train_data = train_data
        self.val_data = val_data

    def evaluate_model(self, parameterization):
        try:
            print("Testing config", parameterization)
            trainer = CNNTrainer(self.train_data)
            model = trainer.build_model(parameterization) 
        except Exception as e:
            return {'acc': 0} 
        
        print("CONFIG Valida")
        model = trainer.fit(model, epochs=self.epochs)
        predictor = CNNPredictor(model)
        _, accuracy = predictor.predict(self.val_data)
        print("ACC during eval", accuracy)
        # Return the validation accuracy as the objective value to optimize
        return {'acc': accuracy}

    def optimize(self):
        
        constraints = ["num_filters1 <= num_filters2",    
                       "num_filters2 <= num_filters3",   
                       "filter_size1 >= filter_size2",   
                       "filter_size2 >= filter_size3",   
                       # "pool_size1 <= pool_size2",   
                       # "pool_size2 <= filter_size2"
                      ]    

        best_parameters, best_values, experiment, model = optimize(
            parameters=self.search_space,
            evaluation_function=self.evaluate_model,
            parameter_constraints=constraints,
            objective_name='acc',
            minimize=False,
            total_trials=self.steps
        )

        print('Best parameters:', best_parameters)
        print('Best validation accuracy:', best_values[0])
        
        return best_parameters, best_values, experiment, model

In [300]:
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Load the CIFAR10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Define the search space

def subsample(data, p=0.3):
    mask = np.random.rand(len(data)) <= p
    mask = np.array(range(len(data)))[mask]
    subset = torch.utils.data.Subset(data, mask)
    return subset

sub_train  = subsample(train_dataset)
sub_val  = subsample(val_dataset)

search_space = [
    {"name": "num_filters1", "type": "range", "bounds": [12, 32], "value_type":"int"},
    {"name": "filter_size1", "type": "range", "bounds": [3, 5], "value_type":"int"},
    {"name": "num_filters2", "type": "range", "bounds": [12, 32], "value_type":"int"},
    {"name": "filter_size2", "type": "range", "bounds": [3, 5], "value_type":"int"},
    {"name": "num_filters3", "type": "range", "bounds": [12, 32], "value_type":"int"},
    {"name": "filter_size3", "type": "range", "bounds": [3, 5], "value_type":"int"},
]
# Initialize the CNNOptimizer
optimizer = CNNOptimizer(search_space, sub_train, sub_val, steps=10, epochs=5)

# Run the optimization
best_parameters, best_values, experiment, model = optimizer.optimize()


Files already downloaded and verified
Files already downloaded and verified


[INFO 03-24 16:29:18] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='num_filters1', parameter_type=INT, range=[12, 32]), RangeParameter(name='filter_size1', parameter_type=INT, range=[3, 5]), RangeParameter(name='num_filters2', parameter_type=INT, range=[12, 32]), RangeParameter(name='filter_size2', parameter_type=INT, range=[3, 5]), RangeParameter(name='num_filters3', parameter_type=INT, range=[12, 32]), RangeParameter(name='filter_size3', parameter_type=INT, range=[3, 5])], parameter_constraints=[OrderConstraint(num_filters1 <= num_filters2), OrderConstraint(num_filters2 <= num_filters3), OrderConstraint(filter_size2 <= filter_size1), OrderConstraint(filter_size3 <= filter_size2)]).
[INFO 03-24 16:29:18] ax.modelbridge.dispatch_utils: Using Models.GPEI since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 03-24 16:29:18] ax.modelbridge.dispatch_utils: Calculating the number

Testing config {'num_filters1': 12, 'filter_size1': 5, 'num_filters2': 18, 'filter_size2': 5, 'num_filters3': 27, 'filter_size3': 4}
Testing config {'num_filters1': 20, 'filter_size1': 4, 'num_filters2': 22, 'filter_size2': 4, 'num_filters3': 23, 'filter_size3': 3}
CONFIG Valida
Running epoch  0
ACC for epoch 0:  0.29643926788685526
Running epoch  1
ACC for epoch 1:  0.4206322795341098
Running epoch  2
ACC for epoch 2:  0.4620299500831947
Running epoch  3
ACC for epoch 3:  0.4961730449251248
Running epoch  4
ACC for epoch 4:  0.5249251247920134


[INFO 03-24 16:29:55] ax.service.managed_loop: Running optimization trial 3...
[INFO 03-24 16:29:55] ax.service.managed_loop: Running optimization trial 4...
[INFO 03-24 16:29:55] ax.service.managed_loop: Running optimization trial 5...


ACC during eval 0.5178511845178512
Testing config {'num_filters1': 18, 'filter_size1': 4, 'num_filters2': 18, 'filter_size2': 4, 'num_filters3': 27, 'filter_size3': 4}
Testing config {'num_filters1': 18, 'filter_size1': 4, 'num_filters2': 18, 'filter_size2': 4, 'num_filters3': 21, 'filter_size3': 4}


[INFO 03-24 16:29:55] ax.service.managed_loop: Running optimization trial 6...
[INFO 03-24 16:29:56] ax.service.managed_loop: Running optimization trial 7...
[INFO 03-24 16:29:56] ax.service.managed_loop: Running optimization trial 8...


Testing config {'num_filters1': 15, 'filter_size1': 5, 'num_filters2': 22, 'filter_size2': 5, 'num_filters3': 26, 'filter_size3': 4}
Testing config {'num_filters1': 16, 'filter_size1': 5, 'num_filters2': 20, 'filter_size2': 5, 'num_filters3': 27, 'filter_size3': 3}
Testing config {'num_filters1': 13, 'filter_size1': 4, 'num_filters2': 17, 'filter_size2': 4, 'num_filters3': 28, 'filter_size3': 4}
Testing config {'num_filters1': 17, 'filter_size1': 4, 'num_filters2': 17, 'filter_size2': 4, 'num_filters3': 21, 'filter_size3': 3}
CONFIG Valida
Running epoch  0
ACC for epoch 0:  0.2849251247920133
Running epoch  1
ACC for epoch 1:  0.41710482529118137
Running epoch  2
ACC for epoch 2:  0.4523793677204659
Running epoch  3
ACC for epoch 3:  0.47966722129783695
Running epoch  4
ACC for epoch 4:  0.5023627287853577


[INFO 03-24 16:30:45] ax.service.managed_loop: Running optimization trial 9...
[INFO 03-24 16:30:45] ax.service.managed_loop: Running optimization trial 10...


ACC during eval 0.5055055055055055
Testing config {'num_filters1': 12, 'filter_size1': 5, 'num_filters2': 21, 'filter_size2': 4, 'num_filters3': 21, 'filter_size3': 4}
Testing config {'num_filters1': 19, 'filter_size1': 4, 'num_filters2': 19, 'filter_size2': 4, 'num_filters3': 30, 'filter_size3': 4}
Best parameters: {'num_filters1': 20, 'filter_size1': 4, 'num_filters2': 22, 'filter_size2': 4, 'num_filters3': 23, 'filter_size3': 3}
Best validation accuracy: {'acc': 0.5178511845178512}


In [301]:
best_parameters, best_values

({'num_filters1': 20,
  'filter_size1': 4,
  'num_filters2': 22,
  'filter_size2': 4,
  'num_filters3': 23,
  'filter_size3': 3},
 ({'acc': 0.5178511845178512}, {'acc': {'acc': nan}}))

In [302]:
data = experiment.fetch_data()
data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,acc,0.000000,NaN,0
1,1_0,acc,0.517851,NaN,1
2,2_0,acc,0.000000,NaN,2
3,3_0,acc,0.000000,NaN,3
4,4_0,acc,0.000000,NaN,4
5,5_0,acc,0.000000,NaN,5
6,6_0,acc,0.000000,NaN,6
7,7_0,acc,0.505506,NaN,7
8,8_0,acc,0.000000,NaN,8
9,9_0,acc,0.000000,NaN,9


In [261]:
trainer = CNNTrainer(train_dataset)
model = trainer.build_model(best_parameters) 
model = trainer.fit(model, epochs=50)


Running epoch  0
Running epoch  1
Running epoch  2
Running epoch  3
Running epoch  4
Running epoch  5
Running epoch  6
Running epoch  7
Running epoch  8
Running epoch  9
Running epoch  10
Running epoch  11
Running epoch  12
Running epoch  13
Running epoch  14
Running epoch  15
Running epoch  16
Running epoch  17
Running epoch  18
Running epoch  19
Running epoch  20
Running epoch  21
Running epoch  22
Running epoch  23
Running epoch  24
Running epoch  25
Running epoch  26
Running epoch  27
Running epoch  28
Running epoch  29
Running epoch  30
Running epoch  31
Running epoch  32
Running epoch  33
Running epoch  34
Running epoch  35
Running epoch  36
Running epoch  37
Running epoch  38
Running epoch  39
Running epoch  40
Running epoch  41
Running epoch  42
Running epoch  43
Running epoch  44
Running epoch  45
Running epoch  46
Running epoch  47
Running epoch  48
Running epoch  49


In [262]:
predictor = CNNPredictor(model)
_, accuracy = predictor.predict(val_dataset)

In [263]:
accuracy

0.6977

In [240]:
# unique, counts = np.unique(labels, return_counts=True)

In [237]:
counts

array([1472, 1493, 1496, 1443, 1500, 1504, 1443, 1547, 1453, 1463])